In [265]:
import numpy as np
import pandas as pd
import datetime
from geopy.geocoders import Nominatim
import pgeocode
import re
import copy
import math

from ipywidgets import IntProgress
from IPython.display import display

import haversine as hs
from haversine import Unit

In [ ]:
data = pd.read_csv("./Data/Исторические совершенные закупки товаров.csv")
data.head()

In [ ]:
template = pd.DataFrame(columns = ['Клиент', 'Материал', 'Краткий текст материала', 'ЕИ',
                                   'Общее количество', 'Месяц поставки', 'Год поставки',
                                   'Полугодие', 'Срок поставки', 'Грузополучатель', 'Цена',
                                   'Способ закупки', '№ заказа', '№ позиции', 'Дата заказа',
                                   'ID Лота'])
template

In [ ]:
data_columns = ['Клиент', 'Материал', 'Материал Имя', 'АЕИ заказа', 'Кол-во к закупу, АЕИ', 
                'Срок поставки', 'Срок поставки', 'Срок поставки', 'Срок поставки', 'Грузополучатель', 
                'Пл.цена с НДС за АЕИ', 'ГПЗ Способ закупки', 'Заявка на закупку', 'Позиц.', 
                'Дата заявки', 'ID лота']

In [ ]:
for col_data, col_temp in zip(data_columns, template.columns):
    template[col_temp] = data[col_data]

In [ ]:
template.head()

In [ ]:
template['Месяц поставки'] = pd.DatetimeIndex(template['Срок поставки']).month
template['Год поставки'] = pd.DatetimeIndex(template['Срок поставки']).year
template['Полугодие'] = (template['Месяц поставки'] > 6) + 1

In [ ]:
grouped_template = template.groupby(by = [pd.DatetimeIndex(template['Дата заказа']).year, 
                                          pd.DatetimeIndex(template['Дата заказа']).month])

In [ ]:
for key, item in grouped_template:
    #print(key, item)
    date = str(key[0])+str(key[1])
    date = datetime.datetime.strptime(date, '%Y%m')
    #item['Дата заказа'] = [date]*item.shape[0]
    item.reset_index(drop = True)
    item.to_csv('./Data/data/'+str(key[0])+'-'+str(key[1])+'.csv', index = False)

# Задача конвертации адресов в координаты и поиска расстояний между ними

In [288]:
geolocator = Nominatim(user_agent="Tester") #Указываем название приложения (так нужно, да)
adress = str(input('Введите адрес: \n')) #Получаем интересующий нас адрес
location = geolocator.geocode(adress) #Создаем переменную, которая состоит из нужного нам адреса
print(location) #Выводим результат: адрес в полном виде
print(location.latitude, location.longitude) #И теперь выводим GPS-координаты нужного нам адреса

Введите адрес: 
 россия москва


Москва, Центральный федеральный округ, Россия
55.625578 37.6063916


In [ ]:
postal_codes = pgeocode.Nominatim('RU')
print(postal_codes.query_postal_code("796902"))

In [ ]:
def find_by_postal_code(postal_code, country = 'RU'):
    """
    postal code: str
    country: str, eg 'RU', 'US' (see pgeocode)
    returns latitude and longitude or None
    """
    #print('find_by_postal_code')
    postal_codes = pgeocode.Nominatim(country)
    ans = postal_codes.query_postal_code(postal_code)
    if ans is None:
        return ans
    lat, lon = ans['latitude'], ans['longitude'] 
    return [lat, lon]
    

In [ ]:
def find_by_special_name(words, names_list, ident = -1):
    """
    words: the list of separated words where to find name
    names_list: list of possible special names
    ident: where to search for the name: before or after the special word, +1 or -1 
    or any possible integer. The program will return all the words in the gap,
    for example for ident = -2 two words will be returned
    returns the whole name without the special word
    """
    #print('find_by_special_name')
    for name in names_list:
        if name in words:
            if name == 'м':
                print(">", words)
            i = words.index(name)
            left, right = min(i, i+ident if i+ident >= 0 else 0), max(i, i+ident)
            query = []
            while left < right:
                query.append(words[left])
                left += 1
            if name == 'м':
                print('<', query)
            return ''.join(query)
    return None

In [ ]:
def location_by_query(query):
    #print('location_by_query', query)
    geolocator = Nominatim(user_agent="Tester")
    location = geolocator.geocode(query)
    if location is not None:
        location = str(location).split(' ')
        return [location[-2], location[-1]]
    return None

In [ ]:
def check_by_special_name(words, names, ident = -1):
    """
    words: an adress word by word
    names: special names, names[0] is the main word used for the location query
    ident: -||-
    returns location or None
    """
    #print('check_by_special_name', words, names)
    ans = find_by_special_name(words, names)
    if ans is not None:
        query = 'Россия' + ', ' + names[0] + '. ' + ans
        ans = location_by_query(query)
        return ans

In [ ]:
def find_coords(adress):
    """
    adress: str in a free form
    returns: [latitude, longitude]
    """
        
        
    # Возможно, у нас будет несколько вариантов для координат
    lat, lon, postal_code = 0, 0, 0
    
    #Давайте попытаемся разбить адрес на "слова"
    adr = adress.lower()
    words = re.findall('\w+', adr)
    #print(words)
    
    #Теперь погнали искать почтовый индекс РФ - 6 цифр
    for el in words:
        if el.isdigit() and len(el) == 6:
            postal_code = el
            ans = find_by_postal_code(postal_code)
            # Если мы нашли координаты по почтовому индексу, 
            # мы вряд ли найдем что-то лучше
            if ans[0] is not None and ans[1] is not None:
                return ans
            
    # Почтовый индекс мы не нашли, или по нему ничего нет
    # Дела наши плохи и плачевны, будем тыркать систему запросами
    special_words = ['г', 'город',
                     'ул', 'улица',
                     'обл', 'область',
                     'с.', 'село',
                     'ст', 'станция',
                     'пос', 'поселок', 'посёлок', 'п',
                     'м/р', 'месторождение',
                     'пгт',
                     'порт', 'аэропорт']
    q_words = ['Россия']

    # Для начала давайте попробуем найти по городу
    for ident in [-2, -1, 1, 2]:
        ans = check_by_special_name(words, ['г', 'город'], ident)
        if ans is not None:
            return ans
        
    # Теперь по поселкам
    ans = check_by_special_name(words, ['п', 'пос', 'поселок', 'посёлок'], 1)
    if ans is not None:
        return ans
    
    # Теперь месторождения
    for ident in [-2, -1, 1, 2]:
        ans = check_by_special_name(words, ['месторождение', 'м'], ident)
        if ans is not None:
            return ans    
        
    # Теперь нам бы хотя бы область узнать
    for ident in [-2, -1, 1, 2]:
        ans = check_by_special_name(words, ['обл', 'область'], ident)
        if ans is not None:
            return ans
    
    # Станция?
    for ident in [-2, -1, 1, 2]:
        ans = check_by_special_name(words, ['станция', 'ст'], ident)
        if ans is not None:
            return ans
        
    return None

In [ ]:
coords = pd.DataFrame(columns = ['Код грузополучателя', 'Адрес грузополучателя', 'Долгота', 'Широта'])
problematic = []
bar = IntProgress(min=0, max=adresses.shape[0]) # instantiate the bar
display(bar) # display the bar

for line in adresses['Адрес грузополучателя']:
    res = find_coords(line)
    if res is not None:
        coords.loc[-1] = [0, line, res[1], res[0]]
        coords.index = coords.index + 1
        #coords = coords.sort_index()
    else:
        problematic.append(line)
    bar.value += 1 # signal to increment the progress bar

In [ ]:
problematic

In [ ]:
coords

In [ ]:
re.findall('\w+', 'Новогоднее м/р, ПС-35/6кВ К-95 ')

In [ ]:
coords.to_csv('./Data/coords_temp.csv', index = False)

# Класс GeoSolver()

In [298]:
class GeoSolver():
    
    """
    Определяет координаты места по адресу, записанному
    в произвольном виде, а также расстояние между двумя
    точками. Работает только для России и +- нормально
    отработает только на данных для нашей конкретной 
    задачи.
    """
    
    def __init__(self):
        self.__geolocator = Nominatim(user_agent="Tester")
        self.__postal_codes = pgeocode.Nominatim('RU')
        self.__special_words = [ 'г', 'город',
                                 'пос', 'поселок', 'посёлок', 'п',
                                 'с', 'село',
                                 'ст', 'станция',
                                 'м/р', 'месторождение',
                                 'пгт',
                                 'порт', 'аэропорт',
                                 'район',
                                 'обл', 'область', 
                                 'регион', 'р', 'округ', 'республика', 'респ'
                                 ]
        
    def find_distance(self, loc1, loc2):
        """
        (широта, долгота)
        возвращает расстояние по прямой в км
        """
        return hs.haversine(loc1, loc2, unit=Unit.KILOMETERS)
    
    
    def check_by_special_name(self, words, name, ident = -1):
        """
        words: список из отдельных слов, составляющих адрес, без пробелов и знаков препинания
        names: опозновательные слова: "г", "город", и т.д.
        ident: в каких пределах ищем имя
        return: позицию по широте и долготе или None
        """
        #print('check_by_special_name', words, names)
        ans = self.find_by_special_name(words, name, ident)
        #print(ans)
        if len(ans) > 0:
            for q in ans:
                query = 'Россия' + ', ' + name + ' ' + q
                #print(f'check: {query}')
                ans = self.location_by_query(query)
                if ans is not None:
                    return ans
        
        
    def find_by_special_name(self, words, name, ident = -1):
        """
        words: the list of separated words where to find name
        names_list: list of possible special names
        ident: where to search for the name: before or after the special word, +1 or -1 
        or any possible integer. The program will return all the words in the gap,
        for example for ident = -2 two words will be returned
        returns the whole name without the special word
        """
        #print('find_by_special_name')
        #print(f'names_list: {name}, {ident}')
        query = []
        for i, el in enumerate(words):
            if name == el:
                #print(f'>found {name} in words')
                q = []
                #print(f'i: {i}, ident = {ident}')
                left, right = min(i+1 if i+1 < len(words) else len(words)-1, (i+ident if i+ident >= 0 else 0)), \
                              max(i-1 if i-1 >= 0 else 0, (i+ident if i+ident < len(words) else len(words)-1))
                #print(f'>> [{left}; {right}]')
                while left < right:
                    q.append(words[left])
                    left += 1
                #print(q)
                if len(q) > 0:
                    query.append(''.join(q))
        return query
    
    
    def location_by_query(self, query):
        #print('location_by_query', query)
        try:
            location = self.__geolocator.geocode(query)
            if location is not None:
                return [location.latitude, location.longitude]
        except:
            print(f'Ошибка! {query}')
        return None
    
    
    def find_by_postal_code(self, postal_code, country = 'RU'):
        """
        postal code: str
        country: str, eg 'RU', 'US' (see pgeocode)
        returns latitude and longitude or None
        """
        #print('find_by_postal_code')
        ans = None
        try:
            ans = self.__postal_codes.query_postal_code(postal_code)
        except:
            print(f'Ошибка! postal_code: {postal_code}')
        if ans is None:
            return ans
        lat, lon = ans['latitude'], ans['longitude'] 
        return [lat, lon]
    
    
    def find_coords(self, adress):
        """
        adress: строка. Адрес записан в произвольном формате
        returns: [широта, долгота]
        """
        #print(adress)

        # Возможно, у нас будет несколько вариантов для координат
        lat, lon, postal_code = 0, 0, 0

        #Давайте попытаемся разбить адрес на "слова"
        adr = adress.lower()
        words = re.findall('\w+', adr)
        #print(words)

        #Теперь погнали искать почтовый индекс РФ - 6 цифр
        #Делаем это в первую очередь из-за вида данных
        for el in words:
            if el.isdigit() and len(el) == 6:
                postal_code = el
                ans = self.find_by_postal_code(postal_code)
                # Если мы нашли координаты по почтовому индексу, 
                # мы вряд ли найдем что-то лучше
                if (ans[0] is not None) and \
                    (ans[1] is not None) and \
                    (not np.isnan(ans[0]) and not np.isnan(ans[1])):
                    return ans, 1.

        # Почтовый индекс мы не нашли, или по нему ничего нет
        # Дела наши плохи и плачевны, будем тыркать систему запросами

        # Для начала давайте попробуем найти по ключевым словам в порядке
        # убывания точности запроса
        for name in self.__special_words:
            #print(f'------- ident in find_coords: {i}')
            for i in [-2, -1, 1, 2]:
                ans = self.check_by_special_name(words, name, i)
                if ans is not None:
                    return ans, 0.5
                
        # Дела совсем плохи, давайте попробуем найти по отдельным
        # словам, может что-то получится
        # Идем с конца, так как более точные данные обычно
        # лежат в конце
        for el in words[::-1]:
            ans = self.location_by_query(el)
            if ans is not None:
                return ans, 0.
                
        
        return None
    
    
    def pd_calc(self, data):
        """
        Обрабатывает данные из pd.DataFrame и возвращает таблицу с доп. колонками
        широтой и долготой, заточена иключительно под нашу задачу (т.е один конкретный
        файл с грузополучателями)
        """
        #coords = pd.DataFrame(columns = ['Код грузополучателя', 'Адрес грузополучателя', 'Долгота', 'Широта'])
        problematic = []
        coords = []
        bar = IntProgress(min=0, max=adresses.shape[0]) # instantiate the bar
        display(bar)                                    # display the bar

        for line, code in zip(data['Адрес грузополучателя'], data['Код грузополучателя']):
            res = self.find_coords(line)
            if res is not None:
                if np.isnan(res[0][0]) or np.isnan(res[0][1]):
                    print(f'WARNING! NaN: {line}')
                coords.append([code, line, res[0][1], res[0][0], res[1]])
            else:
                problematic.append(line)
            bar.value += 1 # signal to increment the progress bar
        
        coords = pd.DataFrame(coords, columns = ['Код грузополучателя', 
                                                 'Адрес грузополучателя', 'Долгота', 
                                                 'Широта', 'Уверенность'])
        coords.to_csv('./Data/coords_test.csv', index = False)
        return problematic

In [299]:
adresses = pd.read_excel('./Data/Справочник грузополучателей.xlsx')
adresses.head()

,Код грузополучателя,Адрес грузополучателя
0,2773,"644099, Россия, г. Омск. ул. Фрунзе, д.54"
1,8536,"РФ, г. Омск, пр. Губкина, д.1"
2,8782,"г. Екатеринбург, ул. Сакко и Ванцетти, д.61"
3,23908,"Россия, 190900, Санкт-Петербург, box 1283"
4,24446,"г. Москва, Капотня, 2 квартал, д.1, корп. 3"


In [300]:
geo_solver = GeoSolver()

In [301]:
geo_solver.find_coords('г. Москва, Капотня, 2 квартал, д.1, корп. 3' )

([43.3620656, 132.8319624], 0.5)

# Тестируем код

In [302]:
p = geo_solver.pd_calc(adresses)

IntProgress(value=0, max=1856)

Ошибка! Россия, г ленинградская
Ошибка! Россия, р 14
Ошибка! Россия, п новый


In [239]:
p

[]

In [297]:
geo_solver.find_distance((55, 37), (60, 30))

694.9292230470902

Поток мысли, который к вск преобразуется в нормальный файл. Сейчас из полезного там реализован класс GeoSolver, который умеет:
* брать адрес и выдавать ~координаты и "уверенность" в них
* пройтись по датафрейму на основе таблицы с грузополучателями и выдать свою таблицу с колонками "код...", "адрес...", "долгота", "широта", "уверенность".
* искать расстояние между двумя точками по их координатам (широта, долгота)

Как работает уверенность (и код): т.к. адреса внесены в общем случае вообще произвольным образом, первым делом координаты ищутся по почтовому индексу. Не-None результату присваивается уверенность 1, т.к. лучше, скорее всего, уже не найти.


Если ПИ отсутствует или не находится, происходит поиск ключевых слов (г, город, п, пос, респ, ...) в порядке "увеличения масштаба". В окрестности ключевого слова ищется название населенного пункта/местности/... из 1-2 слов. Уверенность 0.5 для всех результатов, т.к. geolocator сумасбродничает и на запросы "россия москва" и "россия г москва" выдает разные ответы. Если поиск по ключевому слову не удался, в geolocator закидывается по одному слову из всего запроса, чтобы найти хоть что-то, тут уже уверенность 0.

! табличку грузополучателей код обрабатывает минут 5-7!